# Junção CSV's Civic

Infelizmente, os dados que a API oferece não são suficientes para a realização da nossa análise. Por isso, usaremos dados fornecidos pelo CIVIC em formato TSV, que precisam ser convertidos para CSVs para que possamos trabalhar com esses dados. Assim como na API, os arquivos TSV também estão separados e precisam ser mesclados. Para isso, iremos utilizar a função "merge", que é usada para combinar dois ou mais conjuntos de dados com base em uma chave comum. Para essa etapa, precisamos apenas da importação do Pandas.

In [ ]:
#Importação
import pandas as pd

A função "merge" receberá como entrada dois dataframes, "df_Assertion" e "df_Evidences", e retornará um novo dataframe que contém os dados combinados. Como mencionado anteriormente, os dados estão em formato TSV, mas foram convertidos para CSV com o uso de uma ferramenta externa. Abaixo, definiremos os arquivos que iremos utilizar.

In [ ]:
#Definiçao dos arquivos que serão utilizados
df_Variant = pd.read_csv('01-Apr-2022-VariantSummaries.csv')
df_Evidences = pd.read_csv('01-Apr-2022-ClinicalEvidenceSummaries.csv')
df_Gene = pd.read_csv('01-Apr-2022-GeneSummaries.csv')
df_Assertion = pd.read_csv('01-Apr-2022-AssertionSummaries.csv')

Possuímos quatro DataFrames diferentes: df_Variant, que possui **1533** linhas e **33** colunas, df_evidences, que possui **3897** linhas e **42** colunas, df_gene com **477** linhas e **7** colunas e, por último, df_Assertion com **22** linhas e **28** colunas.

Antes de podermos realizar o merge, precisamos fazer um pequeno tratamento nos dados fornecidos. Inicialmente, precisamos trocar o nome de duas colunas do df_Assertion, que possuem os mesmos valores e significado no df_evidences, porém com nomes diferentes. Para isso, usaremos o método "rename".

In [ ]:
#Trocar o nome das colunas.
df_Assertion.rename(columns={'assertion_type': 'evidence_type'}, inplace = True)
df_Assertion.rename(columns={'assertion_direction': 'evidence_direction'}, inplace = True)

Após trocarmos os nomes dessas colunas, precisamos apagar algumas outras do mesmo df, pois essas colunas não se encontram em nenhum outro df, e por conta disso, a mesclagem se torna um problema. Por isso, essas colunas estão sendo excluídas, mas nenhum dado importante será perdido.

In [ ]:
#Apagando colunas
df_Assertion = df_Assertion.drop(['acmg_codes', 'amp_category', 'nccn_guideline', 
                                  'fda_companion_test', 'assertion_summary', 'assertion_description',
                                  'assertion_id', 'fda_companion_test', 'regulatory_approval',
                                  'assertion_civic_url', 'evidence_item_ids', 'nccn_guideline_version', 'evidence_items_civic_url' ],axis=1)

Com as colunas apagadas, podemos realizar o primeiro merge. Inicialmente, iremos juntar o df_Assertion tratado e o df_evidences, ambos com a maior quantidade de linhas e colunas. Para essa mesclagem, não é necessário passar uma chave específica, podemos simplesmente usar o método "how='outer'". Assim, criaremos um df nomeado Civic que possui a junção de todos os registros dos dois conjuntos de dados originais, mantendo os valores nulos onde não há correspondência entre as chaves de junção, por esse motivo as colunas foram excluídas e tiveram seus nomes trocados.

Usaremos esse método, pois desejamos manter todas as informações de ambos os DataFrames. Caso esse não fosse o objetivo, poderíamos ter usado outros parâmetros, como "inner", "left" ou "right".

In [ ]:
#Merge entre df_Assertion e df_Evidences
Civic = pd.merge(df_Assertion, df_Evidences, how='outer')

Com a primeira junção finalizada, realizaremos a segunda juntando o novo DataFrame nomeado Civic com o df_Variant. Para que essa mesclagem funcione, usaremos como chave a coluna 'variant_id' e como parâmetro usaremos o left_on e o right_on, fazendo o uso desses parâmetros teremos especificado as colunas que serão utilizadas como chaves de junção para o conjunto de dados.

Para que nossos dados não fiquem poluídos e que não precisemos ter um trabalho extra excluindo colunas repetidas que serão geradas com o merge, pois nosso conjunto de dados possuem colunas com o mesmo nome, iremos definir o parâmetro "suffixes" como uma tupla vazia e uma tupla com o sufixo "_y", o que garantirá que não haverá colunas duplicadas após a junção.

Após a junção, usaremos o método "filter" para selecionar apenas as colunas que possuem o sufixo "_y" no nome e usaremos o método "drop" para removê-las.

In [ ]:
#Merge entre Civic e df_Variant
Civic = pd.merge(Civic, df_Variant,  left_on='variant_id', right_on='variant_id', how='left', suffixes=('','_y'))
Civic.drop(Civic.filter(regex='_y$').columns, axis=1, inplace=True)

Com três DataFrames iniciais juntos, seguiremos para o df_genes. Assim como o df_Assertion, ele precisa ser tratado antes da junção, porém por motivos diferentes. O df_genes possui valores duplicados que se encontram em outros DataFrames. Esses valores serão excluídos, e manteremos apenas as colunas "description", que não possuímos em outros DataFrames, e "name", que usaremos como chave de junção.

In [ ]:
#Apagando colunas
df_Gene = df_Gene.drop(['gene_civic_url', 'gene_id', 'last_review_date', 'is_flagged'],axis=1)

Também não possuímos a coluna "name" em nenhum outro dataframe, mas seus valores correspondem às colunas denominadas como "gene" em outros dataframes, por isso faremos um rename nesse dataframe.


In [ ]:
#Trocando o nome da coluna name para gene.
df_Gene.rename(columns={'name': 'gene'}, inplace = True)

Com o "df_gene" tratado, realizaremos o último merge juntando o conjunto total ao último dataframe. Usaremos as colunas "gene" como chave de junção e passaremos o mesmo parâmetro "suffixes" para apagar colunas duplicadas.

In [ ]:
#Merge entre Civic e df_gene
Civic = pd.merge(Civic, df_Gene,  left_on='gene', right_on='gene', how='left', suffixes=('','_y'))
Civic.drop(Civic.filter(regex='_y$').columns, axis=1, inplace=True)

Agora, com o dataframe pronto, realizaremos o último tratamento dessa etapa. Aqui, apagaremos colunas sem valor para nossa análise, como URLs e a última edição ("last_review_date"), colunas com valores nulos em todos os dataframes e colunas que não serão usadas.

In [ ]:
#Removendo colunas
Civic = Civic.drop(['gene_civic_url', 'Unnamed: 32', 'Unnamed: 31', 'Unnamed: 29',
                    'Unnamed: 30', 'assertion_civic_urls', 'variant_civic_url', 
                    'evidence_civic_url', 'last_review_date', 'asco_abstract_id',
                    'nct_ids', 'chromosome2', 'start2', 'stop2', 'representative_transcript2',
                    'variant_groups'
],axis=1)
Civic

Para que o dataframe "Civic" final possa ser utilizado pelo restante do grupo, iremos salvá-lo como CSV.

In [ ]:
#Salvando o df Civic como um csv
Civic.to_csv('Civic.csv', index=False)

Diante disso, podemos ver que nosso código consiste na leitura de quatro arquivos csv (df_Variant, df_Evidence, df_Gene, Df_Assertion) fornecidos pelo proprio Civic, e na sua mesclagem em um dataframe final chamado Civic, tambem possuimos alguns tratamentos sendo realizados nos dados, como a exclusão de colunas não relevantes, a renomeação de colunas e a remoçao de valores duplicados.
Ao final o Dataframe Civic é salvo em um arquivo csv para ser utilizado posteriormente.